In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import daolib.dao as dao
import daolib.dso as dso
import util.sectool as sectool
import util.operatetool as optool
import matplotlib.pyplot as plt
import pickle
import time
import os
%matplotlib inline
from tqdm import tqdm

关于 ['emotion', 'pricevol', 'mom', 'finance', 'valuation', 'alpha191','growth']   六大类因子组合测试（共计150个因子+1个标签值）

导入Miya_Test 封装文件进行测试

In [ ]:
import Miya_Test as miya

### 数据读取

In [3]:
alpha191=readdir('D:\Data\\factors\\alpha191' )
emotion=readdir('D:\Data\\factors\\emotion' )
finance=readdir('D:\Data\\factors\\finance' )
growth=readdir('D:\Data\\factors\\growth' )
mom=readdir('D:\Data\\factors\\mom' )
pricevol=readdir('D:\Data\\factors\\pricevol' )
valuation=readdir('D:\Data\\factors\\valuation' )

stock_price_df = dao.get_security_info('stock_price_info')
all_stock_list = stock_price_df.index.get_level_values(0).drop_duplicates().tolist()

#股票上市状态(只选择为0的值，不为0的值为异常值)
price_df=dao.get_security_info('stock_price_info')
trade_status_df=price_df.xs('trade_status',level=1)

start_date = dt.datetime(2007, 1, 1)
end_date = dt.datetime(2018,10,1)

In [4]:
factor_name_list = ['emotion', 'pricevol', 'mom', 'finance', 'valuation', 'alpha191','growth']  

#周频数据
trade_date_w_series = pd.Series( dso.get_trade_date(start_date=start_date, end_date=end_date, period='w') )
week_list = [date for date in trade_date_w_series if dt.datetime(2009,10,20)<date and date < dt.datetime(2018,11,1) ]
week_data = read_factor_data(factors_name_list=factor_name_list, trade_date_20d_series=trade_date_w_series)
week_data['trade_status'] = trade_status_df

In [5]:
len(week_data)

150

In [8]:
 train1, test1=miya.data_input_class(week_data,week_list, 0.3, -1)

100%|████████████████████████████████████████████████████████████████████████████████| 451/451 [01:03<00:00,  8.76it/s]


In [9]:
test1

,em_cov_volume_1m,em_cov_volume_3m,em_ret_amt_10d,em_ret_amt_1m,em_ret_amt_3d,em_ret_amt_3m,em_ret_amt_5d,em_ret_amt_6m,em_sl_volume_1m,em_turn_cov_1m,...,gw_roe_chg,gw_roe_deducted_chg,gw_roe_deducted_yoy,gw_roe_yoy,gw_yoy_qfa_deductedprofit,gw_yoy_qfa_np_belongto_parcomsh,gw_yoy_qfa_roe_deducted,trade_status,rise_fall,date
000001.SZ,-0.324132,-2.049594,0.389232,0.465589,0.436767,0.480669,0.389232,0.517023,1.582404,-0.365469,...,0.337609,0.166564,0.800036,0.756733,0.001416,0.539291,0.331431,0.0,-1.0,2009-10-23
000002.SZ,0.368445,1.314874,-0.279958,-0.920368,0.008246,-1.044409,-0.279958,-1.018561,-0.212161,0.392372,...,-0.040896,-0.475362,-0.088619,-0.067952,-0.468331,-1.040288,-0.626859,0.0,-1.0,2009-10-23
000004.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000005.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000006.SZ,0.941826,0.842423,-0.395261,-0.747023,-0.709410,-0.916992,-0.395261,-0.872056,0.278021,0.850375,...,0.990950,0.084404,-0.172142,0.536776,0.354234,1.608968,0.467571,0.0,-1.0,2009-10-23
000007.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000008.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000009.SZ,0.597012,0.339801,-0.335572,-0.846388,-0.272695,-0.842423,-0.335572,-0.825869,0.795334,0.490541,...,0.657770,0.674490,0.054270,0.002829,-1.278339,-1.487923,-1.242308,0.0,1.0,2009-10-23
000010.SZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,-1.0,2009-10-23
000012.SZ,-0.107315,-2.124001,0.271259,0.362496,-0.176821,0.289543,0.271259,0.257583,-0.179591,-0.129728,...,1.162040,1.016446,1.299939,1.223293,0.755886,0.608885,0.868783,0.0,-1.0,2009-10-23


### 预测

In [8]:
#分类模型
from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

model_xgb = xgb.XGBClassifier(learning_rate=0.05, max_depth=5, )
model_lgb = lgb.LGBMClassifier(num_leaves=50,learning_rate=0.01)

C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
import Miya_Test as miya

C:\Users\yuanruyi\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
Wfactor0_lgb3,Wfactor1_lgb3,Wtest_accuracy_lgb3,Wtrain_accuracy_lgb3=miya.predict_probo_class(week_data,model_lgb,48,week_list,0.3,-1)
Wfactor0_lgb4,Wfactor1_lgb4,Wtest_accuracy_lgb4,Wtrain_accuracy_lgb4=miya.predict_probo_class(week_data,model_lgb,48,week_list,0.4,-1)

Wfactor0_xgb4,Wfactor1_xgb4,Wtest_accuracy_xgb4,Wtrain_accuracy_xgb4=miya.predict_probo_class(week_data,model_xgb,48,week_list,0.4,-1)
Wfactor0_xgb3,Wfactor1_xgb3,Wtest_accuracy_xgb3,Wtrain_accuracy_xgb3=miya.predict_probo_class(week_data,model_xgb,48,week_list,0.3,-1)

100%|████████████████████████████████████████████████████████████████████████████████| 451/451 [05:51<00:00,  1.34it/s]
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-pac

C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\yuanruyi\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning 

### 因子保存

In [ ]:
import pickle
Wfactor1_lgb3.to_pickle("./factor_test_data/LightGBM_allfactor2/150Wfactor1_lgb3")
Wfactor1_lgb4.to_pickle("./factor_test_data/LightGBM_allfactor2/150Wfactor1_lgb4")
Wfactor1_xgb4.to_pickle("./factor_test_data/LightGBM_allfactor2/150Wfactor1_xgb4")
Wfactor1_xgb3.to_pickle("./factor_test_data/LightGBM_allfactor2/150Wfactor1_xgb3")

### 因子分析

In [ ]:
Wfactor1_lgb3_obj=factor_analyse("test1",Wfactor1_lgb3)
Wfactor1_lgb4_obj=factor_analyse("test1",Wfactor1_lgb4)

Wfactor1_xgb3_obj=factor_analyse("test1",Wfactor1_xgb3)
Wfactor1_xgb4_obj=factor_analyse("test1",Wfactor1_xgb4)

Wsamew_lgb3_pre,Wsamew_lgb3_unpre,Wunw_lgb3_pre,Wunw_lgb3_unpre=factor_stock_choose(Wfactor1_lgb3,100)
Wsamew_lgb4_pre,Wsamew_lgb4_unpre,Wunw_lgb4_pre,Wunw_lgb4_unpre=factor_stock_choose(Wfactor1_lgb4,100)

Wsamew_xgb3_pre,Wsamew_xgb3_unpre,Wunw_xgb3_pre,Wunw_xgb4_unpre=factor_stock_choose(Wfactor1_xgb3,100)
Wsamew_xgb4_pre,Wsamew_xgb4_unpre,Wunw_xgb4_pre,Wunw_xgb4_unpre=factor_stock_choose(Wfactor1_xgb4,100)

In [ ]:
show2(Wfactor1_lgb3_obj)

In [ ]:
show02(Wsamew_lgb3_pre)

### 读取文件夹数据

In [1]:
def readdir(dirname):  #读取文件名
    alist=[]
    for maindir, subdir, file_name_list in os.walk(dirname):
        for filename in file_name_list:
            apath = os.path.join(maindir, filename)
            file_path = os.path.split(apath) #分割出目录与文件  
            lists = file_path[1].split('.') #分割出文件与文件扩展名
            alist.extend(lists)
    return alist

def read_factor_data(factors_name_list, trade_date_20d_series):
    compound_factor_dict = {}
    alpha_factor_series = pd.Series()
    for compound_name in factors_name_list[:]:
        pro_factor_path = "D:\\Data\\factors\\" + compound_name + '\\'
        #alpha_factor_series = pd.Series()
        factor_list = globals().get(compound_name)
        for factor_name in factor_list:
#             print('读取 ', factor_name)
            data_df = pd.read_pickle(pro_factor_path + factor_name)
            begin_date = max(data_df.columns[0], trade_date_20d_series[0])
            trade_date_series = trade_date_20d_series[trade_date_20d_series >= begin_date]
            alpha_factor_series.loc[factor_name] = data_df.loc[:, trade_date_series]
    #alpha_factor_df = ftool.factor_class_generate(alpha_factor_series)
        #compound_factor_dict[compound_name] = alpha_factor_df
    return alpha_factor_series